In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
# Make sure 'count' is included in the import statement
from snowflake.snowpark.functions import col, count, lit, sum
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest


# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="Could not find the number of physical cores for the following reason:*",
    category=UserWarning,
    module="joblib"
)


In [ ]:
# query github evets wide
gh_events_wide_table_name = "GITHUB_EVENTS_WIDE"
gh_events_wide_df = session.table(gh_events_wide_table_name)
global_events_table_name = "GLOBAL_EVENTS"
global_events_df = session.table(global_events_table_name)
actor_table_name = "ACTOR"
actor_df = session.table(actor_table_name)
repo_table_name = "REPO"
repo_df = session.table(repo_table_name)
org_table_name = "ORG"
org_df = session.table(org_table_name)
# concatenate all the dfs in a list to generalize data analysisimport warnings
names_fds = [(gh_events_wide_table_name, gh_events_wide_df), 
       (global_events_table_name, global_events_df),
       (actor_table_name, actor_df),
       (repo_table_name, repo_df),
       (org_table_name, org_df)
      ]

Data exploration

In [ ]:
# for each df print desc
for name, df in names_fds:
    print(f"Description: {name} table:")
    print(df.describe().show())
    print('columns/features schema:')
    print(df.printSchema())
    print('number of rows:')
    print(df.count())

    print('===================================================')


It seems GLOBAL_EVENTS and ACTOR are the most informative data. Focus on them:
- GLOBAL_EVENTS:
    - ID: Event id. non-informative
    - TYPE: closed vocabulary, define the action.
    - ACTOR_ID: Used to merge with Actor.
    - REPO_ID: non-informative in this context.
    - ORG_ID: Id, could be used as a boolean when is missing.
    - PUBLIC: All are TRUE, non-informative.
    - CREATED_AT: Datetime, we could use the day of the week o hour.

- ACTOR
    - ID: used to connect with GLOBAL_EVENTS.
    - LOGIN: custom name
    - GRAVATAR_ID: empty, no-informative.
    - URL: All distinct, non-informative.
    - AVATAR_URL: All distinct, non-informative